In [1]:
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from exp.nb_callbacks import *
from exp.nb_utils import listify

In [96]:
#export
class Learner:
    _default_cbs = [TrainEvalCallback]
    
    ALL_CBS = {
    'begin_fit', 'begin_epoch', 'begin_batch', 'begin_validate'
    'after_pred', 'after_loss', 'after_backward', 'after_step',
    'after_batch', 'after_epoch', 'after_fit',
    'after_cancel_batch', 'after_cancel_epoch', 'after_cancel_train'}

    
    def __init__(self, model, data, loss_func, opt_func, lr=1e-2, cbs=None, cb_funcs=None):
        self.model, self.data, self.loss_func, self.opt_func, self.lr = model, data, loss_func, opt_func, lr
        self.opt = None
        self.cbs = []
        self.add_cbs([cb() for cb in self._default_cbs])
        self.add_cbs(cbs)
        self.add_cbs([cbf() for cbf in listify(cb_funcs)])
        
        self.logger = print
        
    def set_logger(self, logger):    
        self.logger = logger
        
    def add_cb(self, cb):
        cb.set_learner(self)
        setattr(self, cb.name, cb)
        self.cbs.append(cb)
        
    def add_cbs(self, cbs):
        for cb in listify(cbs): self.add_cb(cb)
        
    def remove_cbs(self, cbs):
        for cb in listify(cbs): self.cbs.remove(cb)
    
    def get_params(self):
        return self.model.parameters()

    def _do_begin_fit(self, epochs):
        self.epochs = epochs
        self.loss = tensor(0.)
        self('begin_fit')
        
    def _do_begin_epoch(self, epoch):
        self.epoch = epoch
        self.dl = self.data.train_dl
        return self('begin_epoch')
    
    def one_batch(self, i, xb, yb):
        try:
            self.iter = i
            self.xb, self.yb = xb, yb;                              self('begin_batch')
            self.pred = self.model(self.xb);                        self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb);         self('after_loss')
            # If validate
            if not self.in_train: return
            self.loss.backward();                                   self('after_backward')
            self.opt.step();                                        self('after_step')
            self.opt.zero_grad();
        except CancelBatchException:                                self('after_cancel_batch')
        finally:                                                    self('after_batch')
                                
    
    def all_batches(self):
        self.iters = len(self.dl)
        try:
            for i, (xb, yb) in enumerate(self.dl): 
                self.one_batch(i, xb, yb);
        except CancelEpochException: self('after_cancel_epoch')
    
    def _end_cleanup(self):
        self.dl,self.xb,self.yb,self.pred,self.loss = None,(None,),(None,),None,None

    
    def fit(self, epochs, cbs=None, reset_opt=False):
        self.add_cbs(cbs)
        if reset_opt or not self.opt: 
            self.opt = self.opt_func(self.get_params(), lr=self.lr)
            
        try:
            self._do_begin_fit(epochs)
            for epoch in range(epochs):
                if not self._do_begin_epoch(epoch): 
                    print('heeee')
                    self.all_batches()
                
                with torch.no_grad():
                    self.dl = self.data.valid_dl
                    if not self('begin_validate'): self.all_batches()
                self('after_epoch')
                
        except: CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.remove_cbs(cbs)
            self._end_cleanup()
    
    def __call__(self, cb_name):
        res = False
        assert cb_name in self.ALL_CBS
        for cb in sorted(self.cbs, key=lambda x: x._order):
            res = cb(cb_name) and res
        return res

!python notebook2script.py learner.ipynb

In [97]:
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds[0][0].shape[0]
    print(data.train_ds[0][0].shape)
    model = nn.Sequential(nn.Conv2d(m,nh, 7, 2), nn.ReLU(),
                          nn.Conv2d(nh,nh*2, 5, 2), nn.ReLU(),
                          nn.Conv2d(nh*2,nh*2*2, 3, 2), nn.ReLU(),
                          nn.Conv2d(nh*4,nh*8, 3, 2), nn.ReLU(),
                          nn.AdaptiveAvgPool2d(1),
                          nn.Flatten(), nn.Linear(400,data.c))
    return model

In [98]:
from exp.nb_dataset import get_skin_databunch
db = get_skin_databunch()

In [99]:
mdl = get_model(db)

torch.Size([3, 450, 600])


In [93]:
def accuracy(inp, targ, axis=-1):
    pred,targ = flatten_check(inp.argmax(dim=axis), targ)
    return (pred == targ).float().mean()


In [100]:
learner = Learner(mdl, db, F.cross_entropy, optim.SGD, cb_funcs=[LR_Find, Recorder, ProgressCallback])

In [101]:
learner.fit(1)

heeee
he
heheheheh
wtf


In [102]:
learner.recorder.losses

[tensor(0.)]